In [1]:
# Importar librerías de manejo de datos y preprocesamiento
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# Importar librerías de visualización
import matplotlib.pyplot as plt

# Importar TensorFlow y Keras para construcción de modelos
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation

# Importar módulos de transformers (si estás utilizando modelos preentrenados como BERT)
from transformers import TFBertForSequenceClassification, BertTokenizer

# Importar el optimizador AdamW (si lo estás utilizando en lugar del optimizador Adam de Keras)
from transformers import AdamW

# Otras utilidades de Keras y TensorFlow
from tensorflow.keras.callbacks import EarlyStopping

# Medir el tiempo (si estás evaluando la duración del entrenamiento)
import time


In [2]:
FICHERO_DATA = 'data/data_reviews.csv'

In [3]:
df = pd.read_csv(FICHERO_DATA)

In [4]:
df.head()

,reseñas,IA,longitud,longitud_promedio,palabras_unicas,signos_de_puntuación,frecuencia_pronombres,variedad_lexica,entropia_lexica,expresiones,tfidf,palabras_complejas,tecnicismos,polaridad,subjetividad,coherencia
0,Necesitaba un nuevo movil y me decidí por este...,0,297,4.066667,0.733333,6,0,0.733333,5.267482,0,5.640761e-02,3,0,0.0,0.0,0.920000
1,Está en perfectas condiciones la batería al 10...,0,99,5.375000,1.000000,2,0,1.000000,4.000000,0,5.551115e-17,2,0,0.0,0.0,1.000000
2,Me llegó ayer el iPhone en perfectas condicion...,0,436,4.402439,0.695122,7,0,0.707317,5.710972,0,5.749596e-02,5,0,0.0,0.0,0.857143
3,Compré un iPhone 12 a este vendedor y hace un...,0,657,4.124031,0.581395,10,1,0.604651,5.880116,0,7.866846e-02,5,0,0.0,0.0,0.823529
4,Soy usuario de IPhone desde el principio. Teng...,0,596,4.109244,0.638655,11,0,0.655462,5.953860,0,7.357145e-02,4,0,0.0,0.0,0.953488


In [5]:
# Creamos los datos de entrenamiento
X = df.drop(columns=['reseñas', 'IA'])
y = df[['IA']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Convertimos los datos de entrenamiento en datos estandarizados (media 0, desviación típica 1)
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Definir el modelo (MLP, Multi-Layer Perceptron), modelo relativamente simple

model = Sequential() # Crea un modelo secuencial, que es un contenedor lineal donde puedes apilar capas de una en una.
                     #Este tipo de modelo es adecuado para la mayoría de las redes neuronales feedforward.

# Añadimos una capa densa totalmente conectada con 128 neuronas y la función de activación ReLU.
model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))

# Añadimos una capa dropuot (el 50% de las neuronas se desactiva aleatoriamente)
model.add(Dropout(0.5))  # Regularización para evitar el sobreajuste

model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Salida binaria (0 o 1)
# Esta capa devuelve un valor entre 0 y 1, lo que es adecuado para problemas de clasificación binaria. En este caso, 
# 1 indica que la reseña fue generada por una IA, y 0 indica que no lo fue.

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2, batch_size=32)


Epoch 1/10


c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6369 - loss: 0.6401 - val_accuracy: 0.7962 - val_loss: 0.4740
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7945 - loss: 0.4592 - val_accuracy: 0.7997 - val_loss: 0.4580
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7649 - loss: 0.4870 - val_accuracy: 0.8082 - val_loss: 0.4495
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7871 - loss: 0.4552 - val_accuracy: 0.8151 - val_loss: 0.4435
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7966 - loss: 0.4335 - val_accuracy: 0.8202 - val_loss: 0.4493
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.7988 - loss: 0.4529 - val_accuracy: 0.8168 - val_loss: 0.4398
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - accuracy: 0.7860 - loss: 0.4434 - val_accuracy: 0.8168 - val_loss: 0.4380
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7992 - loss: 0.4328 - val_accuracy: 0.8219 - val_loss: 0.4404

In [8]:
# Vamos a intentar hacerlo algo mas complejo 
model = Sequential()

# Primera capa oculta
model.add(Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(BatchNormalization())  # Normalización por lotes para estabilizar el entrenamiento
model.add(Dropout(0.5))  # Dropout para evitar el sobreajuste

# Segunda capa oculta
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Tercera capa oculta
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Cuarta capa oculta
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Capa de salida
model.add(Dense(1, activation='sigmoid'))  # Salida binaria (0 o 1)

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.1, batch_size=32)


c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5448 - loss: 0.9031 - val_accuracy: 0.7363 - val_loss: 0.6192
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6549 - loss: 0.6835 - val_accuracy: 0.7774 - val_loss: 0.5462
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7031 - loss: 0.6144 - val_accuracy: 0.8082 - val_loss: 0.4991
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7316 - loss: 0.5475 - val_accuracy: 0.8253 - val_loss: 0.4614
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7334 - loss: 0.5512 - val_accuracy: 0.8288 - val_loss: 0.4438
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7447 - loss: 0.5304 - val_accuracy: 0.8219 - val_loss: 0.4389
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7514 - loss: 0.5358 - val_accuracy: 0.8253 - val_loss: 0.4218
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7526 - loss: 0.5202 - val_accuracy: 0.8288 - val_loss:

In [9]:
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {accuracy}')

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - accuracy: 0.7716 - loss: 0.4605
Test Accuracy: 0.7958903908729553


In [10]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Realizar predicciones en el conjunto de prueba
y_pred = (model.predict(X_test_scaled) > 0.5).astype('int32')

# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

# Generar un informe de clasificación
report = classification_report(y_test, y_pred)
print('Classification Report:')
print(report)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Confusion Matrix:
[[294  96]
 [ 53 287]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.75      0.80       390
           1       0.75      0.84      0.79       340

    accuracy                           0.80       730
   macro avg       0.80      0.80      0.80       730
weighted avg       0.80      0.80      0.80       730



In [11]:
model.save('data/modelo_MLP.h5')